In [139]:
# Standard library imports
import os
import datetime as dt

# Data handling and numerical computations
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from scipy import interpolate

import torch
from torch.utils.data import TensorDataset, DataLoader


def smooth(df, n=7):
    smoothed = pd.DataFrame(index = df.index[n:], 
                            columns = df.columns, 
                            data = np.mean(np.asarray([df[i:-(n-i)] for i in range(n)]), 0))
    return smoothed       

def get_state_query_data(num, root = '../google_queries/', append = 'state_queries_new', ignore = [], return_all = False, smooth_after = False):
    state_codes = {'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA':'California','CO':'Colorado','CT':'Connecticut','DE':'Delaware','DC':'District of Columbia','GA':'Georgia','HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'}
    
    code = list(state_codes.keys())[num-1]
    
    df = pd.read_csv(root+append +'/'+code+'_query_data.csv', index_col=0, parse_dates=True)
           
    if smooth_after:
        df = smooth(df)
        
    return df    

def get_hhs_query_data(num, root = '../google_queries/', append = 'state_queries_new', ignore = [], return_all = False, smooth_after = False):
    state_pop = pd.read_csv(root + 'state_population_data_2019.csv', index_col = 0)
    state_dict =  {1:['CT', 'ME', 'MT', 'NH', 'RI', 'VT'],
                   2:['NY', 'NJ'],
                   3:['DE', 'MD', 'PA', 'VA', 'WV', 'DC'],
                   4:['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'SC', 'TN'],
                   5:['IL', 'IN', 'OH', 'MI', 'MN', 'WI'],
                   6:['AR', 'LA', 'NM', 'OK', 'TX'],
                   7:['IA', 'KS', 'MO', 'NE'],
                   8:['CO', 'MT', 'ND', 'SD', 'UT', 'WY'],
                   9:['AZ', 'CA', 'HI', 'NV'],
                  10:['AK', 'ID', 'OR', 'WA']}
    
    total_population = sum([state_pop[state_pop['CODE'] == code]['POP'].values[0] for code in state_dict[num]])
    
    dfs = []
    for code in state_dict[num]:
        if code not in ignore:
            population = state_pop[state_pop['CODE'] == code]['POP'].values[0]/total_population
            new_nf = population*pd.read_csv(root+append +'/'+code+'_query_data.csv', index_col=0, parse_dates=True)
            dfs.append(new_nf)
    
    cols = [d.columns for d in dfs]
    common_cols = cols[0]
    for col_list in cols[1:]:
        common_cols = common_cols.intersection(col_list)
    
    idxs = [d.index for d in dfs]
    common_idxs = idxs[0]
    for idx_list in idxs[1:]:
        common_idxs = common_idxs.intersection(idx_list)
    
    df = pd.DataFrame(index = common_idxs, columns = common_cols, data = 0)
        
    for d in dfs:
        df = df+d.loc[df.index, df.columns]

    if smooth_after:
        df = smooth(df)
        
    if return_all:
        return df, dfs
    return df    

def choose_qs(df, daily_ili, region_num, season, n_qs, region='hhs'):
    state_codes = {'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA':'California','CO':'Colorado','CT':'Connecticut','DE':'Delaware','DC':'District of Columbia','GA':'Georgia','HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'}
    queries = df[region_num]

    if region == 'hhs':
        ili = daily_ili['Region '+str(region_num)]
    elif region == 'state':
        ili = daily_ili[state_codes[list(state_codes.keys())[region_num-1]]]
        
    index = daily_ili.index.intersection(queries.index)
    queries = queries.loc[index]
    ili = ili.loc[index]
    
    dates = pd.date_range(dt.date(season-3, 10, 3), dt.date(season,10,1))

    queries_subset = queries.loc[dates].std()
    queries = queries.iloc[:, np.where(queries_subset != 0)[0]]

    corr_df = pd.DataFrame(index=queries.columns,
                 columns=['correlation'],
                 data=[pearsonr(ili.loc[dates].squeeze(), q)[0] for q in
                               queries.loc[dates].values.T])
    scores = pd.read_csv('Data/Similarity_Scores.csv', index_col=0)
    scores['correlation'] = corr_df
    scores = scores.dropna()
    
    for col in scores.columns:
        scores[col] = scores[col] - scores[col].min()
        scores[col] = scores[col] / scores[col].max()
        scores[col] = 1 - scores[col]
    scores['score'] = np.sqrt(np.square(scores).sum(1))
    
    scores = scores.sort_values('score')
    
    query_choice = scores[:n_qs]
    return query_choice.index

def load_ili(location):
    location_dict = {'US':'Data/national_flu.csv',
                     'England':'Data/England_ILIrates.csv',
                     'state':'Data/state_flu.csv',
                     'hhs':'Data/hhs_flu.csv'}
    
    ili = pd.read_csv(location_dict[location], index_col = -1, parse_dates=True)
    if location == 'state' or location =='hhs':
        new_ili = pd.DataFrame()
        for region in ili['region'].unique():
            new_ili[region] = ili[ili['region'] == region]['unweighted_ili']
        ili = new_ili
        ili /= 13
        ili= ili.fillna(0)
        
    if location == 'US':
        # ili[['weighted_ili']].rename(columns = {'weighted_ili':'National'})
        ili = ili[['weighted_ili']]
        ili /= 13
    
    if location == 'England':
        ili['Date'] = [dt.datetime.strptime(d, '%d/%m/%Y')+dt.timedelta(days=3) for d in ili['ISOWeekStartDate'].values]
        ili = ili[['Date', 'RatePer100000']].set_index('Date')
        ili = ili.rename(columns = {'RatePer100000':'National'})

    return ili

def intepolate_ili(ili, fill_1=False):
    dates = np.asarray([ili.index[0] + dt.timedelta(days=i) for i in
                    range((ili.index[-1] - ili.index[0]).days + 1)])

    x = np.linspace(0, 1, ili.shape[0])
    x2 = np.linspace(0, 1, dates.shape[0])
    f = interpolate.interp1d(x, ili.values, axis = 0, kind = 'cubic')

    if not fill_1:
        return pd.DataFrame(index=dates, columns=ili.columns, data=f(x2))
    else:
        return pd.DataFrame(index=dates, columns=ili.columns, data=ili).fillna(-1)

class DataConstructor:
    def __init__(self, test_season, region = 'hhs', window_size = 28, n_queries = 10, gamma = 28, window = 28, lag = 14, n_regions=10, fill_1 = False, root = 'checkpoints/HHS_SIR_Big_new/' ):

        self.lag = lag
        self.window = window
        self.root = root
        self.n_regions = n_regions
        self.test_season = test_season
        self.region = region
        self.window_size = window_size
        self.n_queries = n_queries
        self.gamma = gamma
        self.fill_1 = fill_1

        if region == 'hhs':
            self.n_regions = 10
        elif region == 'state':
            self.n_regions = 49

    def __call__(self, run_backward=False, no_qs_in_output=False):
        state_codes = {'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA':'California','CO':'Colorado',
                       'CT':'Connecticut','DE':'Delaware','DC':'District of Columbia','GA':'Georgia',
                       'HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky',
                       'LA':'Louisiana','ME':'Maine','MD':'Maryland','MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'}
        ili = load_ili(self.region)
        ili = intepolate_ili(ili, fill_1 = fill_1)

        qs_data_dict = {}
        qs_names_dict = {}
        ignore = ['VI', 'PR']

        for i in range(1,1+self.n_regions):
            if self.region == 'hhs':
                qs_data_dict[i] = get_hhs_query_data(i, ignore=ignore, smooth_after = True)
            elif self.region == 'state':
                qs_data_dict[i] = get_state_query_data(i, ignore=ignore, smooth_after = True)

            qs_names_dict[i] = choose_qs(qs_data_dict, ili, i, self.test_season-1, self.n_queries, region = self.region)
            qs_data_dict[i] = qs_data_dict[i].loc[:, list(qs_names_dict[i])]
            qs_data_dict[i] = qs_data_dict[i].div(qs_data_dict[i].max())

        ili = ili.loc[qs_data_dict[i].index[0] : qs_data_dict[i].index[-1]]
        if self.region == 'state':
            ili = ili[list(state_codes.values())]

        scaler = ili.max()*13
        ili = ili.div(ili.max())

        inputs = []
        outputs = []
        dates = []
        for batch in range(self.window+1, ili.shape[0] - (self.gamma)):
            batch_inputs = []
            for i in range(1,1+self.n_regions):
                batch_inputs.append(qs_data_dict[i].iloc[batch-self.window-1:batch+self.lag-1])
            
            t_ili = ili.iloc[batch-self.window-1:batch+self.lag-1].copy()
            t_ili.iloc[-self.lag:, :] = -1

            batch_inputs.append(t_ili)
            batch_inputs = np.concatenate(batch_inputs, -1)

            
            batch_outputs = []
            for i in range(1,1+self.n_regions):
                if run_backward:
                    batch_outputs.append(qs_data_dict[i].iloc[batch-self.window-1:batch+self.gamma])     
                    t_ili = ili.iloc[batch-self.window-1:batch+self.gamma].copy()
                else:
                    batch_outputs.append(qs_data_dict[i].iloc[batch:batch+self.gamma])            
                    t_ili = ili.iloc[batch:batch+self.gamma].copy()
            
            batch_outputs.append(t_ili)
            batch_outputs = np.concatenate(batch_outputs, -1) 

            if no_qs_in_output:
                batch_outputs = batch_outputs[..., -self.n_regions:]
                
            dates.append((t_ili.index[0]-dt.timedelta(days=1)).to_pydatetime())
            inputs.append(batch_inputs)
            outputs.append(batch_outputs)

        train_test_dates = pd.read_csv('Data/Dates.csv', index_col=0).loc[self.test_season]

        train_start = dt.datetime.strptime(train_test_dates['train_start'], '%Y-%m-%d')
        train_end = dt.datetime.strptime(train_test_dates['train_end'], '%Y-%m-%d')
        test_start = dt.datetime.strptime(train_test_dates['test_start'], '%Y-%m-%d')
        test_end = dt.datetime.strptime(train_test_dates['test_end'], '%Y-%m-%d')

        try:
            train_start = np.where([train_start == d for d in dates])[0][0]
        except:
            train_start = 0
        
        train_end = np.where([train_end == d for d in dates])[0][0]
        test_start = np.where([test_start == d for d in dates])[0][0]
        test_end = np.where([test_end == d for d in dates])[0][0]

        x_train = np.asarray(inputs[train_start:train_end])
        y_train = np.asarray(outputs[train_start:train_end])
        x_test = np.asarray(inputs[test_start:test_end])
        y_test = np.asarray(outputs[test_start:test_end])

        return x_train, y_train, x_test, y_test, scaler
    
def convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=torch.float32):
        x_train = torch.tensor(x_train, dtype = dtype)
        y_train = torch.tensor(y_train, dtype = dtype)
        x_test = torch.tensor(x_test, dtype = dtype)
        y_test = torch.tensor(y_test, dtype = dtype)

        train_dataset = TensorDataset(x_train, y_train)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
        return train_loader


In [96]:
if __name__ == '__main__':
    _data = DataConstructor(test_season=2016, region = 'hhs', window_size=28, n_queries=9, gamma=28)
    x_train, y_train, x_test, y_test, _ = _data()

In [127]:
location = 'US'

In [132]:
location_dict = {'US':'Data/national_flu.csv',
                 'England':'Data/England_ILIrates.csv',
                 'state':'Data/state_flu.csv',
                 'hhs':'Data/hhs_flu.csv'}

ili = pd.read_csv(location_dict[location], index_col = -1, parse_dates=True)
if location == 'state' or location =='hhs':
    new_ili = pd.DataFrame()
    for region in ili['region'].unique():
        new_ili[region] = ili[ili['region'] == region]['unweighted_ili']
    ili = new_ili
    ili /= 13
    ili= ili.fillna(0)
    
if location == 'US':
    ili = ili['weighted_ili']
    # ili[['weighted_ili']].rename(columns = {'weighted_ili':'National'})
    ili /= 13

if location == 'England':
    ili['Date'] = [dt.datetime.strptime(d, '%d/%m/%Y')+dt.timedelta(days=3) for d in ili['ISOWeekStartDate'].values]
    ili = ili[['Date', 'RatePer100000']].set_index('Date')
    ili = ili.rename(columns = {'RatePer100000':'National'})


In [141]:
load_ili('US')

,weighted_ili
week_start,
1997-09-28,0.084729
1997-10-05,0.092313
1997-10-12,0.106058
1997-10-19,0.092246
1997-10-26,0.127398
...,...
2020-08-23,0.077045
2020-08-30,0.070565
2020-09-06,0.079195


In [145]:
train_test_dates = pd.read_csv('Data/Dates.csv', index_col=0)

In [150]:
test_start = dt.datetime.strptime(train_test_dates.loc[2015, 'test_start'], '%Y-%m-%d')
test_end = dt.datetime.strptime(train_test_dates.loc[2015, 'test_end'], '%Y-%m-%d')

,weighted_ili
week_start,
2015-08-30,0.066205
2015-09-06,0.068208
2015-09-13,0.077573
2015-09-20,0.082019
2015-09-27,0.081535
2015-10-04,0.094276
2015-10-11,0.100820
2015-10-18,0.105469
2015-10-25,0.107055


In [171]:
from scipy.stats import gmean

In [136]:
ili = load_ili('US')

In [172]:
for lag in [1,2,3,4]:
    autocorr = pd.DataFrame(index = [2015, 2016, 2017, 2018], columns = ['state', 'hhs', 'US'])
    for season in [2015, 2016, 2017, 2018]:
        test_start = dt.datetime.strptime(train_test_dates.loc[season, 'test_start'], '%Y-%m-%d')
        test_end = dt.datetime.strptime(train_test_dates.loc[season, 'test_end'], '%Y-%m-%d')
        for region in ['state', 'hhs', 'US']:
            ili = load_ili(region)
            ili = ili.loc[test_start:test_end]
            
            # Compute autocorrelation for all columns and store in a new DataFrame
            autocorr_results = pd.DataFrame(index=['autocorr'])
            for column in ili.columns:
                if not np.isnan(ili[column].autocorr()):
                    autocorr_results[column] = ili[column].autocorr(lag = lag)
            
            autocorr_results = autocorr_results.T
            # print(autocorr_results.mean())
    
            autocorr.loc[season, region] = gmean(autocorr_results).item()

    display(autocorr)

/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-pack

,state,hhs,US
2015,0.810441,0.930066,0.95344
2016,0.843001,0.933858,0.963886
2017,0.899874,0.951228,0.968715
2018,0.8819,0.948775,0.965883


/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-pack

,state,hhs,US
2015,0.691289,0.835865,0.868466
2016,0.753258,0.835249,0.888716
2017,0.785778,0.847096,0.885441
2018,0.77495,0.855273,0.88901


/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-pack

,state,hhs,US
2015,0.551381,0.718821,0.76229
2016,0.635841,0.72029,0.785575
2017,0.663391,0.714122,0.769793
2018,0.640237,0.739665,0.790215


/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/data0/mimorris/torch/lib/python3.10/site-pack

,state,hhs,US
2015,0.387529,0.568578,0.625442
2016,0.509458,0.598821,0.671534
2017,0.511503,0.565863,0.633189
2018,NaN,0.610976,0.675965


In [161]:
autocorr.loc[season, region]

,state,hhs,US
2015,0.710288,0.836399,0.868466
2016,0.760659,0.835958,0.888716
2017,0.791813,0.847863,0.885441
2018,0.784443,0.855945,0.88901


In [106]:
autocorr_results

{'Alabama': 0.9328161356660474,
 'Alaska': 0.8699698461677982,
 'Arizona': 0.9475584412319107,
 'Arkansas': 0.9143484256732253,
 'California': 0.9408048681125751,
 'Colorado': 0.9627669154262793,
 'Connecticut': 0.9572684539782085,
 'Delaware': 0.888227057304938,
 'District of Columbia': 0.8894641306628152,
 'Florida': nan,
 'Georgia': 0.9448457941622108,
 'Hawaii': 0.9250866371710722,
 'Idaho': 0.8798663090539671,
 'Illinois': 0.9528789149717682,
 'Indiana': 0.9166022426671017,
 'Iowa': 0.8115362850859874,
 'Kansas': 0.9610733640622768,
 'Kentucky': 0.9580285307469838,
 'Louisiana': 0.9563945483295858,
 'Maine': 0.918491619219672,
 'Maryland': 0.8526173535247382,
 'Massachusetts': 0.9483440978520052,
 'Michigan': 0.9411452021693946,
 'Minnesota': 0.8862675221422462,
 'Mississippi': 0.9385898772638469,
 'Missouri': 0.9369541540186395,
 'Montana': 0.9169062601484708,
 'Nebraska': 0.8993062015856974,
 'Nevada': 0.917564460161929,
 'New Hampshire': 0.9125407600919784,
 'New Jersey': 0.950

In [85]:

dates = np.asarray([ili.index[0] + dt.timedelta(days=i) for i in
                range((ili.index[-1] - ili.index[0]).days + 1)])

x = np.linspace(0, 1, ili.shape[0])
x2 = np.linspace(0, 1, dates.shape[0])
# f = interpolate.interp1d(x, ili.values, axis = 0, kind = 'cubic')

# daily_ili = pd.DataFrame(index=dates, columns=ili.columns, data=f(x2))

In [93]:
df = pd.DataFrame(index=dates, columns=ili.columns, data=ili).fillna(-1)

In [94]:
df

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,New York City,Virgin Islands,Puerto Rico,Commonwealth of the Northern Mariana Islands
2010-10-03,0.164213,0.067319,0.051902,0.053543,0.150317,0.050822,0.006024,0.007702,0.216059,0.0,...,0.113570,0.087956,0.039234,0.122878,0.035771,0.048685,0.090940,0.0,0.0000,0.0
2010-10-04,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2010-10-05,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2010-10-06,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2010-10-07,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-16,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2020-09-17,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2020-09-18,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
2020-09-19,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0000,-1.0
